In [1]:
!pip install prophet xgboost scikit-learn joblib tensorflow statsmodels --quiet


In [2]:
# ===============================================================
# 02_modelling.ipynb - Forecasting Models (Corrected)
# ===============================================================

# STEP 2: Imports
import os, warnings, joblib
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from math import sqrt
from sklearn.metrics import mean_absolute_error, mean_squared_error

warnings.filterwarnings("ignore")
try:
    plt.style.use("seaborn-v0_8")
except:
    plt.style.use("default")

# Models
from statsmodels.tsa.arima.model import ARIMA
from prophet import Prophet
import xgboost as xgb
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import LSTM, Dense


In [3]:
# STEP 3: Ensure models folder exists
os.makedirs("../models", exist_ok=True)

In [4]:
# STEP 4: Load processed dataset from Module 1
df = pd.read_csv("../data/processed/air_quality_cleaned.csv", index_col=0, parse_dates=True)
print("Dataset shape:", df.shape)
print("Columns:", df.columns)
df.head()

Dataset shape: (29531, 18)
Columns: Index(['City', 'PM2.5', 'PM10', 'NO', 'NO2', 'NOx', 'NH3', 'CO', 'SO2', 'O3',
       'Benzene', 'Toluene', 'Xylene', 'AQI', 'AQI_Bucket', 'dayofweek',
       'month', 'year'],
      dtype='object')


,City,PM2.5,PM10,NO,NO2,NOx,NH3,CO,SO2,O3,Benzene,Toluene,Xylene,AQI,AQI_Bucket,dayofweek,month,year
Date,,,,,,,,,,,,,,,,,,
2015-01-01,Ahmedabad,165.63375,309.894886,0.92000,18.220,17.15,4.590000,0.92,27.64000,84.46375,0.00,0.02,0.000000,472.0,NaN,3,1,2015
2015-01-01,Chennai,165.63375,309.894886,16.30000,15.390,22.68,4.590000,1.17,9.20000,11.35000,0.17,12.44,4.920000,472.0,NaN,3,1,2015
2015-01-01,Delhi,165.63375,309.894886,41.05125,36.390,80.18,33.850000,2.82,9.25000,41.68000,7.38,23.37,8.737708,472.0,Severe,3,1,2015
2015-01-01,Lucknow,165.63375,309.894886,2.11000,13.460,4.57,29.353333,2.82,29.49125,25.92000,1.35,3.93,4.920000,467.5,NaN,3,1,2015
2015-01-01,Mumbai,165.63375,309.894886,2.68500,15.395,27.38,24.856667,0.00,29.49125,18.32500,0.00,0.00,0.000000,463.0,NaN,3,1,2015


In [5]:
# STEP 5: Evaluation function
def evaluate(y_true, y_pred):
    if not isinstance(y_pred, pd.Series):
        y_pred = pd.Series(y_pred, index=y_true.index)
    df_eval = pd.concat([y_true, y_pred], axis=1).dropna()
    if df_eval.empty:
        return (np.nan, np.nan)
    y_true_clean, y_pred_clean = df_eval.iloc[:,0], df_eval.iloc[:,1]
    mae = mean_absolute_error(y_true_clean, y_pred_clean)
    rmse = sqrt(mean_squared_error(y_true_clean, y_pred_clean))
    return mae, rmse


In [9]:
# Import needed modules if not done already
from statsmodels.tsa.arima.model import ARIMA
from prophet import Prophet
import xgboost as xgb
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
import numpy as np
import pandas as pd

import joblib

all_results = {}

pollutants = ['PM2.5', 'PM10', 'NO', 'NO2', 'NOx', 'NH3', 'CO', 'SO2', 'O3', 'Benzene', 'Toluene', 'Xylene']

for pollutant in pollutants:
    print("="*60)
    print(f"Training models for {pollutant}")
    print("="*60)

    series = df[pollutant].dropna()

    # Fix: Drop duplicate timestamps indices to avoid reindex errors
    series = series[~series.index.duplicated(keep='first')]  

    # Set daily frequency for time series
    series = series.asfreq("D")

    if len(series) < 100:
        print(f"Skipping {pollutant} (not enough data)")
        continue

    train_size = int(len(series) * 0.8)
    train, test = series[:train_size], series[train_size:]

    results = {}

    # Persistence baseline
    persistence_preds = test.shift(1).fillna(method="bfill")
    results["Persistence"] = evaluate(test, persistence_preds)

    # ARIMA model
    try:
        arima_model = ARIMA(train, order=(5,1,0))
        arima_fit = arima_model.fit()
        arima_forecast = pd.Series(arima_fit.forecast(steps=len(test)), index=test.index)
        results["ARIMA"] = evaluate(test, arima_forecast)
    except Exception as e:
        print("ARIMA failed:", e)
        results["ARIMA"] = (np.nan, np.nan)

    # Prophet model
    try:
        prophet_df = pd.DataFrame({"ds": train.index, "y": train.values})
        prophet_df["ds"] = prophet_df["ds"].dt.tz_localize(None)
        prophet_model = Prophet(daily_seasonality=True, weekly_seasonality=True, yearly_seasonality=True)
        prophet_model.fit(prophet_df)
        future = prophet_model.make_future_dataframe(periods=len(test), freq="D")
        future["ds"] = future["ds"].dt.tz_localize(None)
        forecast = prophet_model.predict(future)
        prophet_forecast = forecast.set_index("ds")["yhat"].iloc[-len(test):]
        prophet_forecast.index = test.index
        results["Prophet"] = evaluate(test, prophet_forecast)
    except Exception as e:
        print("Prophet failed:", e)
        results["Prophet"] = (np.nan, np.nan)

    # XGBoost model
    try:
        def make_lags(series, n_lags=7):
            df_lag = pd.DataFrame({"y": series})
            for lag in range(1, n_lags+1):
                df_lag[f"lag_{lag}"] = df_lag["y"].shift(lag)
            return df_lag.dropna()

        lags = make_lags(series, n_lags=7)
        train_lags = lags.loc[train.index.min():train.index.max()]
        test_lags = lags.loc[test.index.min():test.index.max()]

        X_train, y_train = train_lags.drop(columns="y"), train_lags["y"]
        X_test, y_test = test_lags.drop(columns="y"), test_lags["y"]

        scaler = StandardScaler()
        X_train_s = scaler.fit_transform(X_train)
        X_test_s = scaler.transform(X_test)

        xgb_model = xgb.XGBRegressor(n_estimators=300, learning_rate=0.05, max_depth=5, random_state=42)
        xgb_model.fit(X_train_s, y_train)

        xgb_preds = pd.Series(xgb_model.predict(X_test_s), index=X_test.index)
        results["XGBoost"] = evaluate(y_test, xgb_preds)
    except Exception as e:
        print("XGBoost failed:", e)
        results["XGBoost"] = (np.nan, np.nan)

    # LSTM model
    try:
        def create_sequences(values, n_steps=7):
            X, y = [], []
            for i in range(len(values) - n_steps):
                X.append(values[i:i+n_steps])
                y.append(values[i+n_steps])
            return np.array(X), np.array(y)

        values = series.values
        n_steps = 7
        X_all, y_all = create_sequences(values, n_steps)
        split_i = int(len(X_all) * 0.8)
        X_train_seq, X_test_seq = X_all[:split_i], X_all[split_i:]
        y_train_seq, y_test_seq = y_all[:split_i], y_all[split_i:]

        X_train_seq = X_train_seq.reshape((X_train_seq.shape[0], X_train_seq.shape[1], 1))
        X_test_seq  = X_test_seq.reshape((X_test_seq.shape[0], X_test_seq.shape[1], 1))

        lstm = Sequential()
        lstm.add(LSTM(50, activation="relu", input_shape=(n_steps,1)))
        lstm.add(Dense(1))
        lstm.compile(optimizer="adam", loss="mse")
        lstm.fit(X_train_seq, y_train_seq, epochs=10, verbose=0)

        lstm_preds = lstm.predict(X_test_seq, verbose=0).flatten()
        lstm_index = series.index[-len(y_test_seq):]
        lstm_preds_series = pd.Series(lstm_preds, index=lstm_index)
        results["LSTM"] = evaluate(series.loc[lstm_index], lstm_preds_series)
    except Exception as e:
        print("LSTM failed:", e)
        results["LSTM"] = (np.nan, np.nan)

    # Save best model
    results_df = pd.DataFrame(results, index=["MAE","RMSE"]).T
    all_results[pollutant] = results_df
    best_model = results_df["RMSE"].dropna().idxmin()
    print(f"Best model for {pollutant}: {best_model}")

    # Save models correctly
    if best_model == "ARIMA":
        joblib.dump(arima_fit, f"../models/{pollutant}_arima.pkl")
    elif best_model == "Prophet":
        joblib.dump(prophet_model, f"../models/{pollutant}_prophet.pkl")
    elif best_model == "XGBoost":
        joblib.dump(xgb_model, f"../models/{pollutant}_xgb.pkl")
        joblib.dump(scaler, f"../models/{pollutant}_xgb_scaler.pkl")
    elif best_model == "LSTM":
        lstm.save(f"../models/{pollutant}_lstm.keras")
    elif best_model == "Persistence":
        joblib.dump(series, f"../models/{pollutant}_persistence.pkl")

    print(f"Saved {best_model} model for {pollutant}\n")


Training models for PM2.5


23:25:26 - cmdstanpy - INFO - Chain [1] start processing
23:25:26 - cmdstanpy - INFO - Chain [1] done processing


Best model for PM2.5: LSTM
Saved LSTM model for PM2.5

Training models for PM10


23:25:35 - cmdstanpy - INFO - Chain [1] start processing
23:25:35 - cmdstanpy - INFO - Chain [1] done processing


Best model for PM10: LSTM
Saved LSTM model for PM10

Training models for NO


...

Best model for Benzene: Prophet
Saved Prophet model for Benzene

Training models for Toluene


23:27:07 - cmdstanpy - INFO - Chain [1] start processing
23:27:07 - cmdstanpy - INFO - Chain [1] done processing


Best model for Toluene: LSTM
Saved LSTM model for Toluene

Training models for Xylene


23:27:17 - cmdstanpy - INFO - Chain [1] start processing
23:27:17 - cmdstanpy - INFO - Chain [1] done processing


Best model for Xylene: ARIMA
Saved ARIMA model for Xylene



In [10]:
# STEP 7: Summarize best models
summary = {}
for pollutant, res in all_results.items():
    best = res["RMSE"].dropna().idxmin()
    summary[pollutant] = {
        "Best Model": best,
        "MAE": res.loc[best, "MAE"],
        "RMSE": res.loc[best, "RMSE"]
    }

summary_df = pd.DataFrame(summary).T
summary_df


,Best Model,MAE,RMSE
PM2.5,LSTM,21.569968,29.384597
PM10,LSTM,48.896476,63.847904
NO,LSTM,9.357396,11.629328
NO2,LSTM,12.710144,16.41996
NOx,LSTM,15.892867,20.196671
NH3,ARIMA,10.327133,14.59398
CO,LSTM,0.45242,0.592358
SO2,LSTM,5.861742,7.626662
O3,Prophet,13.982668,17.801481
Benzene,Prophet,1.985413,2.38367
